'''

Algoritmo:
1. Abre o site https://veiculos.fipe.org.br/#carro-comum
2. Extrai os dados de carros, caminhões e motos
3. Exporta para uma planilha com três abas, uma para cada

'''

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from tkinter import messagebox

In [37]:
caminho = r'C:\Users\wellington\Desktop\Suporte Atendimento\projetos\APIFipe\APIFipe\chromedriver-win64\chromedriver.exe'  # Path do chromedriver
servico = Service(caminho)  # Configura o serviço do ChromeDriver
navegador = webdriver.Chrome(service=servico)  # Cria o navegador
navegador.set_window_size(930,1014)  # Define o tamanho da tela

In [41]:
def extrai_veiculos(veiculos:list) -> tuple:
  '''
  Inicia a extração dos veículos da tabela FIPE

  Parâmetros:
    - veiculos: é uma das posições da tabela de veículos da FIPE (carros, caminhões ou motos)
  
  Retorno:
    - tuple: tupla com todos os modelos, anos e montadoras
  '''
  veiculos.click()
  carros = veiculos.find_element(By.CLASS_NAME, 'horizontal')
  artigos_html = carros.find_elements(By.TAG_NAME, 'article')
  formulario = artigos_html[0].find_element(By.CLASS_NAME, 'form')
  step_1 = formulario.find_element(By.CLASS_NAME, 'step-1')
  step_2 = formulario.find_element(By.CLASS_NAME, 'step-2')
  entrada_carros = step_1.find_element(By.CLASS_NAME, 'input')

  sleep(0.8)
  entrada_carros.click()
  
  seletor = entrada_carros.find_element(By.TAG_NAME, 'select')
  opcoes_carros = seletor.find_elements(By.TAG_NAME, 'option')

  lista_montadoras = []
  lista_anos = []
  lista_modelos = []
  for op in opcoes_carros:
    if op.text == '': continue

    lista_montadoras.append(op.text)
    sleep(0.8)
    op.click()
    tupla_listas = seleciona_ano(step_2)
    lista_anos.append(tupla_listas[0])
    lista_modelos.append(tupla_listas[1])
  
  sleep(0.8)
  entrada_carros.click()
  
  return (lista_montadoras, lista_anos, lista_modelos)
  


def seleciona_ano(step_2) -> tuple:
  '''
  Extrai todos os anos da montadora em questão

  Parâmetros:
    - step_2: o step-2 da tabela, onde se localiza a lista de anos e modelos da montadora selecionada

  Retorna:
    - tuple: uma tupla com os modelos e os anos
  '''
  entradas = step_2.find_elements(By.CLASS_NAME, 'input')

  sleep(0.8)
  entradas[1].click()

  seletor = entradas[1].find_element(By.TAG_NAME, 'select')
  opcoes_ano = seletor.find_elements(By.TAG_NAME, 'option')

  lista_anos = []
  lista_modelos = []
  for op in opcoes_ano:
    if op.text == '': continue

    lista_anos.append(op.text)
    sleep(0.8)
    op.click()
    item_modelos = seleciona_modelo(entradas[0])
    lista_modelos.append(item_modelos)
  
  return (lista_anos, lista_modelos)


def seleciona_modelo(entrada) -> list:
  '''
  Extrai todos os modelos do ano em questão

  Parâmetros:
    - entrada: um input do step-2 da tabela, onde se localiza a lista de modelos do ano selecionado

  Retorna:
    - list: uma lista com os modelos do an1.5
  '''
  sleep(0.8)
  entrada.click()

  seletor = entrada.find_element(By.TAG_NAME, 'select')
  opcoes_modelo = seletor.find_elements(By.TAG_NAME, 'option')

  lista_modelos = []
  for op in opcoes_modelo:
    if op.text == '': continue
    
    lista_modelos.append(op.text)

  return lista_modelos

In [42]:
sleep(0.8)

navegador.get('https://veiculos.fipe.org.br/#carro-comum')

messagebox.showinfo(title='Início', message='Pode iniciar?')

tabela_veiculos = navegador.find_element(By.CLASS_NAME, 'tab-veiculos')
veiculos = tabela_veiculos.find_elements(By.CLASS_NAME, 'ilustra')


In [32]:
# CARROS 
tupla_modelos = extrai_veiculos(veiculos[0])

montadoras = tupla_modelos[0]
anos = tupla_modelos[1]
modelos = tupla_modelos[2]

montadoras_lista = []
anos_lista = []
modelos_lista = []
ciclo_montadora = 0
while ciclo_montadora < len(montadoras):
  for index_ano in range(len(anos[ciclo_montadora])):
    for index_mod in range(len(modelos[ciclo_montadora][index_ano])):
      montadoras_lista.append(montadoras[ciclo_montadora])
      anos_lista.append(anos[ciclo_montadora][index_ano])
      modelos_lista.append(modelos[ciclo_montadora][index_ano][index_mod])

  ciclo_montadora += 1

montadoras_ano_carros = {
        'Montadrora': montadoras_lista,
        'Ano': anos_lista,
        'Modelo': modelos_lista
        }

veiculos = pd.DataFrame(montadoras_ano_carros)

veiculos.to_excel('FIPE.xlsx')
messagebox.showinfo(title='Fipe - Carros', message='A Tabela da Fipe Carros foi atualizada!')


'ok'

In [40]:
# CAMINHÕES
tupla_modelos = extrai_veiculos(veiculos[1])

montadoras = tupla_modelos[0]
anos = tupla_modelos[1]
modelos = tupla_modelos[2]

montadoras_lista = []
anos_lista = []
modelos_lista = []
ciclo_montadora = 0
while ciclo_montadora < len(montadoras):
  for index_ano in range(len(anos[ciclo_montadora])):
    for index_mod in range(len(modelos[ciclo_montadora][index_ano])):
      montadoras_lista.append(montadoras[ciclo_montadora])
      anos_lista.append(anos[ciclo_montadora][index_ano])
      modelos_lista.append(modelos[ciclo_montadora][index_ano][index_mod])

  ciclo_montadora += 1

montadoras_ano_carros = {
        'Montadrora': montadoras_lista,
        'Ano': anos_lista,
        'Modelo': modelos_lista
        }

veiculos = pd.DataFrame(montadoras_ano_carros)

veiculos.to_excel('FIPE-Caminhão.xlsx')
messagebox.showinfo(title='Fipe - Caminhões', message='A Tabela da Fipe Caminhões foi atualizada!')

'ok'

In [43]:
# MOTOS
tupla_modelos = extrai_veiculos(veiculos[2])

montadoras = tupla_modelos[0]
anos = tupla_modelos[1]
modelos = tupla_modelos[2]

montadoras_lista = []
anos_lista = []
modelos_lista = []
ciclo_montadora = 0
while ciclo_montadora < len(montadoras):
  for index_ano in range(len(anos[ciclo_montadora])):
    for index_mod in range(len(modelos[ciclo_montadora][index_ano])):
      montadoras_lista.append(montadoras[ciclo_montadora])
      anos_lista.append(anos[ciclo_montadora][index_ano])
      modelos_lista.append(modelos[ciclo_montadora][index_ano][index_mod])

  ciclo_montadora += 1

montadoras_ano_carros = {
        'Montadrora': montadoras_lista,
        'Ano': anos_lista,
        'Modelo': modelos_lista
        }

veiculos = pd.DataFrame(montadoras_ano_carros)

veiculos.to_excel('FIPE-Motos.xlsx')
messagebox.showinfo(title='Fipe - Motos', message='A Tabela da Fipe Motos foi atualizada!')

'ok'

In [30]:
# TODOS JUNTOS
for ciclo in range(3):
  tupla_modelos = extrai_veiculos(veiculos[ciclo])

  montadoras = tupla_modelos[0]
  anos = tupla_modelos[1]
  modelos = tupla_modelos[2]

  montadoras_lista = []
  anos_lista = []
  modelos_lista = []
  ciclo_montadora = 0
  while ciclo_montadora < len(montadoras):
    for index_ano in range(len(anos[ciclo_montadora])):
      for index_mod in range(len(modelos[ciclo_montadora][index_ano])):
        montadoras_lista.append(montadoras[ciclo_montadora])
        anos_lista.append(anos[ciclo_montadora][index_ano])
        modelos_lista.append(modelos[ciclo_montadora][index_ano][index_mod])

    ciclo_montadora += 1

  montadoras_ano_carros = {
          'Montadrora': montadoras_lista,
          'Ano': anos_lista,
          'Modelo': modelos_lista
          }

  veiculos = pd.DataFrame(montadoras_ano_carros)

  if ciclo == 0:
    veiculos.to_excel('FIPE.xlsx')
    messagebox.showinfo(title='Fipe - Carros', message='A Tabela da Fipe Carros foi atualizada!')
  elif ciclo == 1:
    veiculos.to_excel('FIPE-Caminhão.xlsx')
    messagebox.showinfo(title='Fipe - Caminhões', message='A Tabela da Fipe Caminhões foi atualizada!')
  elif ciclo == 2:
    veiculos.to_excel('FIPE-Motos.xlsx')
    messagebox.showinfo(title='Fipe - Motos', message='A Tabela da Fipe Motos foi atualizada!')

navegador.quit()

IndexError: list index out of range